<a href="https://colab.research.google.com/github/kyriosaa/AI-Chatbot/blob/main/AI_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install llmx==0.0.11a0
!pip install openai==0.28.1
!pip install googlesearch-python
!pip install gradio

import openai
openai.api_key = "YOUR API KEY HERE"
import gradio as gr
from googlesearch import search
from IPython.display import Image, display, Markdown

# Functions that can be streamed or non-streamed using the ChatCompletion API
# You can automatically call the function specified by stop_func when getting a full reply
def get_reply_g(messages, stream=False):
    try:
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = messages,
            stream = stream
        )
        if stream: # In the streaming mode, hand it over to the helper function to get the complete text
            for res in response:
                if 'content' in res['choices'][0]['delta']:
                    yield res['choices'][0]['delta']['content']
        else:      # In non-streaming mode, you can get the full reply text directly
            yield response['choices'][0]['message']['content']
    except openai.OpenAIError as err:
        reply = f"Occur {err.error.type} Error\n{err.error.message}"
        print(reply)
        yield reply

# A template to ask if you need to search to answer a question
# Ask AI to reply Y/N  in JSON format along with suggested search keywords
template_google = '''
If I want to know the following, do I need to perform a web search?

```
{}
```

If nessescary, please answer me in the following JSON format. In addition to the JSON format information, do not add additional information. Even if you know the answer, do not reply:

```
{{
    "search":"Y",
    "keyword":"Your suggested search keywords"
}}
```
If not, please answer me in the following JSON format：

```
{{
    "search":"N",
    "keyword":""
}}
'''

# Use the current history and boilerplate content to ask if you need to search to answer your question
# If you need to reply, you can also get the search terms recommended by the AI
def check_google(hist, msg, verbose=False):
  reply = get_reply_g(
    hist + [{  # Add to the history of AI to recommend the right keywords
    "role": "user",
    "content": template_google.format(msg)
  }])
  for ans in reply:pass
  if verbose: print(ans)
  return ans

def google_res(user_msg, num_results=5, verbose=False):
    content = "The following are the facts that happened: \n"                # Emphasize the credibility of the data
    for res in search(user_msg, advanced=True,    # Thread the search results one by one
                      num_results=num_results,
                      lang='en-US'):
        content += f"Title: {res.title}\n" \
                    f"Summary: {res.description}\n\n"
    content += "Please answer the following questions based on the above facts: \n"        # Give clear instructions
    if verbose:
        print('------------')
        print(content)
        print('------------')
    return content

import json
hist = []       # Transcript of previous conversations
backtrace = 2   # Record several sets of conversations

def chat_g(sys_msg, user_msg, stream=False, verbose=False):
    global hist
    messages = [{'role':'user', 'content':user_msg}]
    ans = json.loads(check_google(hist, user_msg,
                                  verbose=verbose))
    if ans['search'] == 'Y':
        print(f'Try searching the web: {ans["keyword"]}....')
        res = google_res(ans['keyword'], verbose=verbose)
        messages = [{'role':'user', 'content': res + user_msg}]

    reply_full = ''
    replies = get_reply_g(            # Use the functions of the search version
        hist        # Historical records are provided first
        + messages  # The search results and current information are then provided
        + [{"role": "system", "content": sys_msg}],
        stream)
    for reply in replies:
        reply_full += reply
        yield reply

    hist += [
        {"role":"user", "content":user_msg},
        {"role":"assistant", "content":reply_full}
    ]
    while len(hist) >= 2 * backtrace: # The record limit is exceeded
        hist.pop(0)                   # Remove the oldest record

func_table = [
    {                       # Each element represents a function
        "chain": True,      # Whether the result of the function execution is to be passed back to the API
        "func": google_res, # Function name
        "spec": {           # Function calling required specifications
            "name": "google_res",
            "description": "Get Google search results",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_msg": {
                        "type": "string",
                        "description": "Keywords to search for",
                    }
                },
                "required": ["user_msg"],
            },
        }
    }
]

# Fetch the function name and argument content from the function_calling object returned from the API, automatically call the function and return the result
def call_func(func_call):
    func_name = func_call['name']
    args = json.loads(func_call['arguments'])
    for f in func_table: # Find the item that contains this function
        if func_name == f['spec']['name']:
            print(f"Try calling：{func_name}(**{args})")
            val = f['func'](**args)
            return val, f['chain']
    return '', False

# Return content from API to find out function_calling content
def get_func_call(messages, stream=False, func_table=None,
                  **kwargs):
    model = 'gpt-3.5-turbo'
    if 'model' in kwargs: model = kwargs['model']
    funcs = {}
    if func_table:
        funcs = {'functions':[f['spec'] for f in func_table]}
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        stream = stream,
        **funcs
    )
    if stream:
        chunk = next(response)
        delta = chunk["choices"][0]["delta"]
        if 'function_call' in delta:
            func_call = delta['function_call']
            args = ''
            for chunk in response:
                delta = chunk["choices"][0]["delta"]
                if 'function_call' in delta:
                    args += delta['function_call']['arguments']
            func_call['arguments'] = args
            return func_call, None
    else:
        msg = response["choices"][0]["message"]
        if 'function_call' in msg:
            return msg['function_call'], None
    return None, response

def get_reply_f(messages, stream=False, func_table=None, **kwargs):
    try:
        func_call, response = get_func_call(messages,
                                            stream, func_table, **kwargs)
        if func_call:
            res, chain = call_func(func_call)
            if chain:  # If necessary, send the result of the function execution back to the AI for the reply
                messages += [
                    {  # The original function_calling must be returned
                        "role": "assistant", "content": None,
                        "function_call": func_call
                    },
                    {  # Function exection result with the function role
                        "role": "function",        # function role
                        "name": func_call['name'], # Returns the name of the function
                        "content": res             # Deliver the execution result
                    }]
                yield from get_reply_f(messages, stream,
                                       func_table, **kwargs)
            else:      # Chain is False, and the result is called the model generated by the function
                yield res
        elif stream:   # You dont need to call a function, you can instead use a streaming pattern
            for chunk in response:
                if 'content' in chunk['choices'][0]['delta']:
                    yield chunk['choices'][0]['delta']['content']
        else:          # There is no need to call a function or use a streaming pattern
            yield response['choices'][0]['message']['content']
    except openai.OpenAIError as err:
        reply = f"Occur {err.error.type} Error\n{err.error.message}"
        print(reply)
        yield reply

hist = []       # Transcript of historical conversations
backtrace = 2   # Record several sets of conversations

def chat_f(sys_msg, user_msg, stream=False, **kwargs):
    global hist

    replies = get_reply_f(    # Use functions from the Functions Feature Edition
        hist                  # Historical records are provided first
        + [{"role": "user", "content": user_msg}]
        + [{"role": "system", "content": sys_msg}],
        stream, func_table, **kwargs)
    reply_full = ''
    for reply in replies:
        reply_full += reply
        yield reply

    hist += [{"role":"user", "content":user_msg},
             {"role":"assistant", "content":reply_full}]
    while len(hist) >= 2 * backtrace: # The record limit is exceeded
        hist.pop(0)  # Remove the oldest record

def txt_to_img_url(prompt):
    response = openai.Image.create(
        prompt=prompt,
        n=1,
        size='1024x1024')
    return response['data'][0]['url']

func_table.append(
    {                    # Each element represents a function
        "chain": False,  # After generating the image, there is no need to send it back to the API
        "func": txt_to_img_url,
        "spec": {        # Function specification required by function calling
            "name": "txt_to_img_url",
            "description": "You can generate an image from text and send it back to the image URL",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "Describe the text that you want to produce the content of the image",
                    }
                },
                "required": ["prompt"],
            },
        }
    }
)

def wrapper_chat(sys_msg, user_msg, stream):
    reply = ''
    for chunk in chat_f(sys_msg, user_msg, stream):
        reply += chunk
        yield reply

web_chat = gr.Interface(
    fn = wrapper_chat,
    inputs = ['text', 'text', 'checkbox'],
    outputs = ['text']
)

messages = [] # Record the complete conversation and display it in the bubble interface

def wrapper_chat_bot(sys_msg, user_msg, stream):
    messages.append([user_msg, '']) # Add questions and empty answers first
    for chunk in chat_f(sys_msg, user_msg, stream):
        messages[-1][1] += chunk # Add accumulated snippets to answers
        yield messages # Return the content of the current answer

web_chat = gr.Interface(
    fn=wrapper_chat_bot,
    inputs=[
        gr.Textbox(label='System role', value='Use English assistant'),
        gr.Textbox(label='User speaks'),
        gr.Checkbox(label='Use streaming', value=False)],
    outputs=[gr.Chatbot(label='AI reply')]
)

def txt_to_img_md(prompt):
    return f'![{prompt}]({txt_to_img_url(prompt)})'

func_table.pop() # Remove mapping functions that return URLs from the reference table

func_table.append({      # Each element represents a function
        "chain": False,  # There is no need to send the image back to the API
        "func": txt_to_img_md,
        "spec": {        # Function specifications required for function calling
            "name": "txt_to_img_md",
            "description": "Can generate images and text and return markdown image",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "Text describing the content of the image to be generated",
                    }
                },
                "required": ["prompt"],
            },
        }
})

web_chat = gr.Interface(
    fn=wrapper_chat_bot,
    inputs=[
        gr.Textbox(label='System role', value='Use English assistant'),
        gr.Textbox(label='User speaks'),
        gr.Checkbox(label='Use streaming', value=False)],
    outputs=[gr.Chatbot(label='AI reply')]
)

web_chat.queue()
web_chat.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cf500741a87890e827.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
